<a href="https://colab.research.google.com/github/Farooqbasha008/Analysis-of-Emotional-Intensity-of-tweets/blob/main/models/Deep_learning_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install tensorflow
!pip install keras-tuner


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 5.1 MB/s eta 0:00:00


In [21]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.metrics import MeanSquaredError
from tensorflow.keras.optimizers import Adam
from kerastuner.tuners import BayesianOptimization
import shutil

# Read the train and test datasets
train_df = pd.read_csv('trained.csv')
test_df = pd.read_csv('tested.csv')

# Drop unnecessary columns
train_df = train_df.drop('Unnamed: 0', axis=1)
test_df = test_df.drop('Unnamed: 0', axis=1)

# Prepare the train and test data
X_train = train_df.iloc[:, :5]
y_train = train_df['score']

X_test = test_df.iloc[:, :5]
y_test = test_df['score']

# Split the data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Define the build_model function
def build_model(hp):
    model = Sequential()

    # Tune the number of layers
    num_layers = hp.Int('num_layers', min_value=1, max_value=5)
    for i in range(num_layers):
        model.add(Dense(units=hp.Int(f'units_{i}', min_value=32, max_value=128, step=32),
                        activation='relu'))

    model.add(Dense(1, activation='linear'))

    model.compile(optimizer=Adam(
                    learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
                  loss='mean_squared_error',
                  metrics=[MeanSquaredError()])

    history = model.fit(X_train, y_train,
                        epochs=hp.Int('epochs', min_value=50, max_value=200, step=50),
                        batch_size=hp.Int('batch_size', min_value=32, max_value=128, step=32),
                        validation_data=(X_val, y_val),
                        verbose=0)

    return model

checkpoint_dir = 'my_dir/emotional_intensity'

# Clear the checkpoint directory if it exists
shutil.rmtree(checkpoint_dir, ignore_errors=True)

# Initialize the Keras Tuner BayesianOptimization tuner
tuner = BayesianOptimization(
    build_model,
    objective='mean_squared_error',
    max_trials=10,
    executions_per_trial=2,
    directory='my_dir',
    project_name='emotional_intensity',
    overwrite = True
)

# Perform the hyperparameter search
tuner.search(X_train, y_train)

# Retrieve the best model and evaluate it
best_model = tuner.get_best_models(num_models=1)[0]
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]

print("Best Hyperparameters:")
for hyperparameter, value in best_hyperparameters.values.items():
    print(f"{hyperparameter}: {value}")

loss = best_model.evaluate(X_test, y_test)
print("Loss:", loss)


Trial 10 Complete [00h 00m 21s]
mean_squared_error: 0.028210964053869247

Best mean_squared_error So Far: 0.01227064710110426
Total elapsed time: 00h 08m 34s
Best Hyperparameters:
num_layers: 5
units_0: 64
learning_rate: 0.0001
epochs: 150
batch_size: 32
units_1: 32
units_2: 32
units_3: 32
units_4: 32
99/99 [==============================] - 0s 1ms/step - loss: 0.0116 - mean_squared_error: 0.0116
Loss: [0.011582504026591778, 0.011582504026591778]


In [22]:
# Build the model with the obtained hyperparameters
model = Sequential()

# Add the layers based on the best hyperparameters
model.add(Dense(units=64, activation='relu', input_dim=X_train.shape[1]))
model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=32, activation='relu'))

# Add the output layer
model.add(Dense(units=1, activation='linear'))

# Compile the model
model.compile(loss='mean_squared_error',
              optimizer=Adam(learning_rate=0.0001),
              metrics=[MeanSquaredError()])

# Train the model
model.fit(X_train, y_train, epochs=150, batch_size=32, validation_data=(X_test, y_test))


# Calculate y_pred
y_pred = model.predict(X_test)

# Flatten y_pred if necessary
y_pred = y_pred.flatten()

# Evaluate the model
loss = model.evaluate(X_test, y_test)
print("Loss:", loss)


# Calculate MAE
mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error (MAE):", mae)

# Calculate MSE
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error (MSE):", mse)

# Calculate RMSE
rmse = np.sqrt(mse)
print("Root Mean Squared Error (RMSE):", rmse)

# Calculate R2 score
r2 = r2_score(y_test, y_pred)
print("R-squared (R2):", r2)

# Calculate RMSLE
rmsle = np.sqrt(mean_squared_log_error(y_test, y_pred))
print("Root Mean Squared Logarithmic Error (RMSLE):", rmsle)

# Calculate MAPE
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
print("Mean Absolute Percentage Error (MAPE):", mape)

Epoch 1/150
79/79 [==============================] - 2s 6ms/step - loss: 0.2780 - mean_squared_error: 0.2780 - val_loss: 0.2254 - val_mean_squared_error: 0.2254
Epoch 2/150
79/79 [==============================] - 0s 4ms/step - loss: 0.1701 - mean_squared_error: 0.1701 - val_loss: 0.1093 - val_mean_squared_error: 0.1093
Epoch 3/150
79/79 [==============================] - 0s 4ms/step - loss: 0.0645 - mean_squared_error: 0.0645 - val_loss: 0.0392 - val_mean_squared_error: 0.0392
Epoch 4/150
79/79 [==============================] - 0s 4ms/step - loss: 0.0381 - mean_squared_error: 0.0381 - val_loss: 0.0379 - val_mean_squared_error: 0.0379
Epoch 5/150
79/79 [==============================] - 0s 4ms/step - loss: 0.0378 - mean_squared_error: 0.0378 - val_loss: 0.0378 - val_mean_squared_error: 0.0378
Epoch 6/150
79/79 [==============================] - 0s 4ms/step - loss: 0.0378 - mean_squared_error: 0.0378 - val_loss: 0.0377 - val_mean_squared_error: 0.0377
Epoch 7/150
79/79 [===============